In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')

# Load Dataset

In [3]:
DATASET_NAME = "Agari" 
FIELDS = ["text"]

In [4]:
import spacy
from notebooks.utils import get_client
dataset = get_client(DATASET_NAME)

# Define Entities

## REGEX Entities

In [5]:
SSN_ENTITY = {"entity": "SSN", "TYPE": "Regex", "pattern": r"[0-9]{3}-[0-9]{2}-[0-9]{4}|(ssn|social security)[^0-9]{0,4}\K([0-9\-]{9,12})"}

## Only if using NER

In [ ]:
nlp = spacy.load('en_core_web_lg',  enable=['ner'])
NER_ENTITY = {
    "entities": ["PERSON", "ORG"], 
    "TYPE": "Model", 
    "model": nlp,
    "preprocess": lambda t: t.replace("\n", ". "),
    "filter": lambda value, entity: value.strip() and value not in ["»"]
}

# Load Extractors

In [6]:
from texas.ml.entities import extract_entities

# Check

In [7]:
TEST_ENTITY = "SSN"
TEST_FIELD = FIELDS[0]

entities_to_extract = [SSN_ENTITY]

for _id, d in extract_entities(dataset.scan(), entities_to_extract, FIELDS):
    if d.get("__nlp__", {}).get(TEST_FIELD, {}).get(TEST_ENTITY, None):
        print("Example for", TEST_ENTITY)
        print(_id)
        for result in d.get("__nlp__", {}).get(TEST_FIELD, {}).get(TEST_ENTITY, []):
            print(result)
        break

2018-03-22 14:35:30,378 INFO GET http://elasticsearch:9200/agari_conversations/email/_search?scroll=5m&size=1000 [status:200 request:0.327s]
2018-03-22 14:35:31,226 INFO DELETE http://elasticsearch:9200/_search/scroll [status:200 request:0.002s]


Example for SSN
bournejason519@gmail.com,michael.mun77@gmail.com
{'value': '563-98-6250', 'span': [185, 196]}
{'value': '092-64-0254', 'span': [650, 661]}
{'value': '574-02-4092', 'span': [1165, 1176]}
{'value': '574-84-8727', 'span': [1614, 1625]}
{'value': '574-56-4331', 'span': [2015, 2026]}
{'value': '601-36-5899', 'span': [2462, 2473]}
{'value': '170-44-3700', 'span': [2922, 2933]}
{'value': '574-90-9752', 'span': [3389, 3400]}
{'value': '227-78-4703', 'span': [3792, 3803]}
{'value': '574-08-3736', 'span': [4261, 4272]}
{'value': '362-88-7385', 'span': [4618, 4629]}
{'value': '574-94-8234', 'span': [5056, 5067]}
{'value': '537-08-0739', 'span': [5499, 5510]}
{'value': '574-26-9327', 'span': [5968, 5979]}
{'value': '433-92-7069', 'span': [6306, 6317]}
{'value': '477-60-7220', 'span': [6767, 6778]}
{'value': '533-68-9264', 'span': [7225, 7236]}
{'value': '537-04-8868', 'span': [7688, 7699]}
{'value': '071-56-9270', 'span': [8108, 8119]}
{'value': '574-80-6103', 'span': [8556, 8567]}

# Process Whole Data

In [ ]:
entities_to_extract = [SSN_ENTITY]
dataset.write(extract_entities(dataset.scan(), entities_to_extract, FIELDS))